In [1]:
import pandas as pd
import jieba
import zhconv
import re

In [2]:
df = pd.read_csv(r'.\data\weibo_senti_100k\weibo_senti_100k.csv')
df.sample(20)

,label,review
69709,0,@黄栌记忆 ：请阅读潘一恒孩子的画，伤痕呵！ // @洁平 : [泪] 香港东方日报：殉职的...
99015,0,我最爱的Hachi！ //@冷笑话精选:哭的最惨的电影 没有之一！！[抓狂]
5287,1,此刻江岭的人与油菜花一样多！ //@山高路远林密:乡村美[赞]//@婺源文风:早安，永忠友[...
38166,1,在某使馆排了两小时队，把我给冻的，腰都站疼了，[晕][抓狂]还好没白排这么长时间。接下来再计...
33811,1,昨天忙了一天，没来得及发微博。可是活动很多。开心又略感伤感的莫过于和好朋友--新华社驻泰记者...
38137,1,@耿长军-小黑 知道你怕拍照，?是昨晚聊天的?候我把手?放在桌面上偷拍的[哈哈]
114736,0,回复@幸运兔子__罗征:全TMD男单。跟演健美比赛是的。 //@幸运兔子__罗征:您了要看A...
32177,1,三个和尚[嘻嘻][嘻嘻][嘻嘻][嘻嘻] 我在:http://t.cn/zjjKdnN
10512,1,#婚庆圣地 蜜月度假#@三亚海棠湾洲际度假酒店 开业之际，携手尔雅凤凰（北京）文化传媒公司推...
61337,0,#豆果分享#【对抗重度污染！我有清肺美食！】[抓狂]拿什么拯救你，我脆弱的肺！最近很多地区陷...


In [3]:
emotion = {0: "负向", 1: "正向"}
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119988 entries, 0 to 119987
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   label   119988 non-null  int64 
 1   review  119988 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.8+ MB


In [4]:
df.label.value_counts()

0    59995
1    59993
Name: label, dtype: int64

In [5]:
# 简繁切换
df["review"] = df["review"].apply(lambda x: zhconv.convert(str(x), 'zh-cn'))

In [6]:
# 读取停用词，stopwords.txt 使用 空格 划分每一个词语
with open("stopwords.txt", "r", encoding="utf-8") as fp:
    stop_word = fp.read().split()
stop_word[: 10]

['!', '"', '#', '$', '%', '&', "'", '(', ')', '*']

In [7]:
def jieba_cut_text(text, stop_word, cut_by=" "):
    # [\u4E00-\u9FA5]+ 去除非中文字符
    patt = re.compile("[\u4E00-\u9FA5]+", re.VERBOSE|re.IGNORECASE)
    word_list = []
    for t in patt.findall(text):
        word_list += [word for word in jieba.cut(t) if word not in stop_word]

    return cut_by.join(word_list)

In [8]:
jieba_cut_text(df.review[10], stop_word)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\11353\AppData\Local\Temp\jieba.cache
Loading model cost 0.613 seconds.
Prefix dict has been built successfully.


'鼓掌 慕春彦 一流 经纪 公司 超模 摇篮 鼓掌 姚戈 东方 宾利 强大 名模 军团'

In [9]:
df["review"] = df["review"].apply(lambda x: jieba_cut_text(x, stop_word))

In [10]:
df["review"].sample(20)

60070     中 册 那妈 逼 日 八哥 丫路 中 港乱 日 八哥 丫路 中 勿要 面孔 瘪三 日 八哥 ...
5032                              高速 省油 特别 公里 数 诞生 留影 纪念 锐志
45499                         知足 李咏 接地 气儿 撮 人儿 法图麦 妈妈 情况 抓狂
95053     摄影 穷 三代 单反 毁 一生 真 说错 年底 借钱 先买 条 皮带 再也 不用 鞋带 紧急...
16352     五月 北京 红墙 碧瓦 绿树 成萌 到来 鼓掌 南平 事 体验 时间 月 日至 月 日 任选...
47167                                                     花
86691                中国 太有钱 孔 宏伟 金佛 中国 国内 牛比 国外 煞笔 俗称 窝里横 怒
54816     回复 金属 之心 过奖 过奖 太 开心 太 开心 太 开心 讨嫌 脑有 包 沐元 文玩 金属...
104439                         这句 话 摇摇 心酸 摇摇 赔给 夜夜 梦 晚安 亲 衰
56552                                         特工 傀儡 梅 姐夫 墨涵
25244                       回复 孤 风中 凌乱 塞维利亚 孤 塞尔维亚 塞利 维亚 思考
66444           回复 活力 兔 泪 活力 兔 怒 蜡烛 午夜 阳光 奇奇 逝者 安息 蜡烛 蜡烛 蜡烛
3864                             抱抱 做做 桦桦 葛芳 时尚 瘦身 身体 生活 更好
20104                                                    现场
3068      围观 鼓掌 鼓掌 鼓掌 张明健 演员 回复 演员 黄河 握手 握手 演员 黄河 恭喜 明键 ...
119334    达人 揪 团 带路 台湾 自由 行 路书 投票 页面 上线 希望 大伙 帮 投票 投 票 爱...
37358                              雪 凝 不

In [11]:
df["label"] = df["label"].apply(lambda x: emotion[x])

In [12]:
df.label.value_counts()

负向    59995
正向    59993
Name: label, dtype: int64

In [13]:
df.to_pickle("weibo_senti_100k_preprocessing.pkl.bz2")